# Pretraining on WT103

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
#from exp.nb_12a import *
from exp.nb_formatted import *

## Data

One time download

In [14]:
#path = datasets.Config().data_path()
path = Path('/mnt/ssd_1TB/Data/fastai/')
version = '103' #2

In [15]:
path

PosixPath('/mnt/ssd_1TB/Data/fastai')

In [7]:
! wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-{version}-v1.zip -P {path}
! unzip -q -n {path}/wikitext-{version}-v1.zip  -d {path}
! mv {path}/wikitext-{version}/wiki.train.tokens {path}/wikitext-{version}/train.txt
! mv {path}/wikitext-{version}/wiki.valid.tokens {path}/wikitext-{version}/valid.txt
! mv {path}/wikitext-{version}/wiki.test.tokens {path}/wikitext-{version}/test.txt

--2019-06-14 22:27:23--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.185.197
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.185.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: '/mnt/ssd_1TB/Data/fastai/wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  5.02MB/s    in 40s     

2019-06-14 22:28:04 (4.58 MB/s) - '/mnt/ssd_1TB/Data/fastai/wikitext-103-v1.zip’ saved [190229076/190229076]



Split the articles: WT103 is given as one big text file and we need to chunk it in different articles if we want to be able to shuffle them at the beginning of each epoch.

In [16]:
#path = datasets.Config().data_path()/'wikitext-103'
path = path/'wikitext-103'

In [17]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

In [18]:
def read_wiki(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            current_article = current_article.replace('<unk>', UNK)
            articles.append(current_article)
            current_article = ''
    current_article = current_article.replace('<unk>', UNK)
    articles.append(current_article)
    return articles

In [19]:
train = TextList(read_wiki(path/'train.txt'), path=path) #+read_file(path/'test.txt')
valid = TextList(read_wiki(path/'valid.txt'), path=path)

In [20]:
len(train), len(valid)

(28476, 60)

In [21]:
sd = SplitData(train, valid)

In [22]:
proc_tok,proc_num = TokenizeProcessor(),NumericalizeProcessor()

In [23]:
ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

In [24]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [25]:
ll = pickle.load( open(path/'ld.pkl', 'rb'))

In [26]:
bs,bptt = 128,70
data = lm_databunchify(ll, bs, bptt)

In [27]:
vocab = ll.train.proc_x[-1].vocab
len(vocab)

60002

## Model

In [28]:
dps = np.array([0.1, 0.15, 0.25, 0.02, 0.2]) * 0.2
tok_pad = vocab.index(PAD)

In [29]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(vocab), emb_sz, nh, nl, tok_pad, *dps)

In [30]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressCallback]

In [31]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [32]:
lr = 5e-3
sched_lr  = combine_scheds([0.3,0.7], cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds([0.3,0.7], cos_1cycle_anneal(0.8, 0.7, 0.8))
cbsched = [ParamScheduler('lr', sched_lr), ParamScheduler('mom', sched_mom)]

In [33]:
learn.fit(10, cbs=cbsched)

epoch,train_loss,train_accuracy_flat,valid_loss,valid_accuracy_flat,time


RuntimeError: CUDA out of memory. Tried to allocate 52.00 MiB (GPU 0; 10.73 GiB total capacity; 125.96 MiB already allocated; 23.06 MiB free; 38.04 MiB cached)

In [ ]:
torch.save(learn.model.state_dict(), path/'pretrained.pth')
pickle.dump(vocab, open(path/'vocab.pkl', 'wb'))